<a href="https://colab.research.google.com/github/KPraveenRaj/IPCV_PROJECT-Implementation/blob/main/IPCV_Project_PFT_SR_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create cache directories in Drive
import os
drive_cache = '/content/drive/MyDrive/Colab_Notebooks/MTech_IPCV/IPCV_Project/PFT_SR_CACHE'
os.makedirs(drive_cache, exist_ok=True)
os.makedirs(drive_cache + '/repo', exist_ok=True)
os.makedirs(drive_cache + '/cuda_ext', exist_ok=True)
print('✅ Drive cache ready:', drive_cache)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive cache ready: /content/drive/MyDrive/Colab_Notebooks/MTech_IPCV/IPCV_Project/PFT_SR_CACHE


In [5]:
import os
repo_path = '/content/drive/MyDrive/Colab_Notebooks/MTech_IPCV/IPCV_Project/PFT-SR'
cache_repo = '/content/drive/MyDrive/Colab_Notebooks/MTech_IPCV/IPCV_Project/PFT_SR_CACHE/repo'

if not os.listdir(cache_repo):
    print('Cloning repo first time...')
    !git clone https://github.com/LabShuHangGU/PFT-SR.git
    !cp -r /content/PFT-SR/* {cache_repo}/
else:
    print('Loading repo from Drive cache...')
    !cp -r {cache_repo} /content/
    !mv /content/repo /content/PFT-SR

%cd /content/PFT-SR

Loading repo from Drive cache...
mv: cannot move '/content/repo' to '/content/PFT-SR/repo': Directory not empty
/content/PFT-SR


# Maually load the Pretrained Models

####Load the pretrained models into "./experiments/pretrained_models" from [here](https://drive.google.com/drive/folders/1ChkxVDghFWUtJydJKLp5yssrUfm0VWfg)

(ignore if you have already loaded the models in the drive cache)

In [6]:
# Install CUDA Toolkit 12.1 (only if cached nvcc does not exist)
if not os.path.exists('/usr/local/cuda-12.1/bin/nvcc'):
    print('Installing CUDA Toolkit 12.1...')
    !wget https://developer.download.nvidia.com/compute/cuda/12.1.0/local_installers/cuda_12.1.0_530.30.02_linux.run
    !sudo sh cuda_12.1.0_530.30.02_linux.run --silent --toolkit
else:
    print('CUDA Toolkit already installed in this session.')

os.environ['CUDA_HOME'] = '/usr/local/cuda-12.1'
os.environ['PATH'] += ':/usr/local/cuda-12.1/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.1/lib64'

!nvcc --version

CUDA Toolkit already installed in this session.
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Feb__7_19:32:13_PST_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0


# After the below pip install restart the runtime

In [ ]:
# Install PyTorch 2.5.0+cu121
!pip install --upgrade --force-reinstall torch==2.5.0+cu121 torchvision==0.20.0+cu121 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu121

In [7]:
import torch
print('CUDA from PyTorch:', torch.version.cuda, 'Available:', torch.cuda.is_available())

CUDA from PyTorch: 12.1 Available: True


In [8]:
!pip install --upgrade addict future lmdb numpy opencv-python Pillow \
        pyyaml requests scikit-image scipy tb-nightly tqdm yapf fairscale

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/smm_cuda-1.0-py3.12-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)


In [9]:
# Build CUDA extension only if not cached
import os
cuda_cache = '/content/drive/MyDrive/Colab_Notebooks/MTech_IPCV/IPCV_Project/PFT_SR_CACHE/cuda_ext'


print('Compiling CUDA extension for the first time...')
!python setup.py develop
%cd ops_smm
!bash make.sh
%cd ..
!cp -r ops_smm {cuda_cache}/


Compiling CUDA extension for the first time...
/usr/local/lib/python3.12/dist-packages/setuptools/__init__.py:80: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.12/dist-packages/setuptools/command/develop.py:42: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/s

In [13]:
from google.colab import files
uploaded = files.upload()
input_image = list(uploaded.keys())[0]
print('Uploaded:', input_image)

Saving sign.jpg to sign.jpg
Uploaded: sign.jpg


In [14]:
!python inference.py -i $input_image -o results/colab_output/ --scale 4 --task lightweight

/usr/local/lib/python3.12/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/content/PFT-SR/inference.py:87: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_

In [16]:
from google.colab import files
import os

base_name = os.path.splitext(os.path.basename(input_image))[0]
output_image = f"{base_name}_PFT_lightweight_SRx4.jpg"
files.download(f'results/colab_output/{output_image}')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>